In [1]:
using GLMakie, FileIO, Colors
using LinearAlgebra
using BenchmarkTools
using StaticArrays
using SatelliteDynamics
using DelimitedFiles
using ForwardDiff

In [2]:
const G = 6.67430e-20 #km**3/(kg * s**2)
const m_1 = 5.97219e24 #mass of the Earth in kg
const m_2 = 1 #mass of satellite in kg
const μ = G*m_1
const R_earth = 6378.12 # in km
h = 1 #time step in seconds

1

In [3]:
distance_scale = R_EARTH*(1e-3) # scales km to custom scale
time_scale = 1/(C_LIGHT/R_EARTH) #scales seconds to custom scale

c = 1 # 1 distance_unit/ 1 time_unit

1

In [4]:
function elevation_angle(satposes, r0)
    
    elevangle = zeros(4)
    #x = 0
    #y = 0

    #A,B = horizon(r0[1:3])
    #z =  A*(x-r0[1]) + B*(y-r0[2]) + r0[3]

    #vector in the horizon plane
    #vec_inplane = [x,y,z] - r0[1:3]

    #vector between tag and the satellite
    normalvec = r0
    
    for i in 1:4
        
        vector = satposes[i,:] - r0[1:3]
        
        #Find the angle between the normal and the vector going from the tag to the satellte
        theta = acos(dot(normalvec,vector)/(norm(normalvec)*norm(vector))) * (180/pi)
        
        #save elevation angle
        elevangle[i] = theta
    end
    
    return elevangle
end

elevation_angle (generic function with 1 method)

In [5]:
function sat_dynamics(x)
    
    sat_pose = SA[x[1], x[2], x[3]]
    
    #This WORKS
    upperleft = @SMatrix zeros(3,3)
    upperright = SMatrix{3,3}(1.0I)
    lowerleft = SMatrix{3,3}(1I)*(-μ/(norm(sat_pose))^3)
    lowerright = @SMatrix zeros(3,3)
    
    
    upper = [upperleft upperright]
    lower = [lowerleft lowerright]
    
    A = [upper; lower]
    
    xdot = A*x
    
    return xdot
    
end

sat_dynamics (generic function with 1 method)

In [6]:
function RK4_orbit(x, h)
    
    f1 = sat_dynamics(x)
    f2 = sat_dynamics(x+0.5*h*f1)
    f3 = sat_dynamics(x+0.5*h*f2)
    f4 = sat_dynamics(x+h*f3)
    
    xnext = x+(h/6.0)*(f1+2*f2+2*f3+f4)

    return xnext
    
end

RK4_orbit (generic function with 1 method)

In [7]:
function twobody_rk4(fun, x0, Tf, h)
    
    t = Array(range(0,Tf, step=h)) #create a range to final time at constant time step
    
    all_x = zeros(length(x0), length(t)) #variable to store all x
    
    all_x[:,1] .= x0 #set the initial state
    
    for k=1:(length(t) - 1)
        all_x[:,k+1] .= RK4_orbit(all_x[:,k], h)
    end
    
    return all_x, t
end

twobody_rk4 (generic function with 1 method)

In [8]:
#Tag position
#equator position in cartesian coordinates
#tag =[-6.128804e6, -1.590206e6, 776.1502e3]
#Equitorial position
tag_geof = [-165.4545, 0, 0]

#North pole position
#tag_geof = [-165.4545, 46.98849, 0]

tag = sGEOCtoECEF(tag_geof, use_degrees = true)
    
#Equitorial Position
#In m
x0= tag[1]*1e-3
y0 = tag[2]*1e-3
z0 = tag[3]*1e-3
t0 = 1e-5

r0 = [x0, y0, z0, t0]
r0_scaled = [x0/distance_scale, y0/distance_scale, z0/distance_scale, t0/time_scale]

4-element Vector{Float64}:
 -0.9679486083847263
 -0.2511487826487322
  0.0
  0.0004700314384940316

In [9]:
function orbit_update(trueanom, RAAN_sep, delta_sep, altitude)
    
    iss1 = [6371e3 + (altitude*1e3), 0.0004879, 90.6391, 194.0859- (RAAN_sep/2), 151.2014, 190];
    iss2 = [6371e3 + (altitude*1e3), 0.0004879, 90.6391, 194.0859 - (RAAN_sep/2), 151.2014, 190+trueanom];
    iss3 = [6371e3 + (altitude*1e3), 0.0004879, 90.6391, 195.0859 + (RAAN_sep/2), 151.2014, 190+delta_sep]; 
    iss4 = [6371e3 + (altitude*1e3), 0.0004879, 90.6391, 195.0859 + (RAAN_sep/2), 151.2014, 190+delta_sep+trueanom]; 
    
    eci0_1 = sOSCtoCART(iss1, use_degrees=true)
    eci0_2 = sOSCtoCART(iss2, use_degrees=true)
    eci0_3 = sOSCtoCART(iss3, use_degrees=true)
    eci0_4 = sOSCtoCART(iss4, use_degrees=true)
    
    T = orbit_period(iss1[1])
    
    #in km
    x0_1 = SA[eci0_1[1], eci0_1[2], eci0_1[3],eci0_1[4], eci0_1[5], eci0_1[6]]*1e-3 
    x0_2 = SA[eci0_2[1], eci0_2[2], eci0_2[3],eci0_2[4], eci0_2[5], eci0_2[6]]*1e-3 
    x0_3 = SA[eci0_3[1], eci0_3[2], eci0_3[3],eci0_3[4], eci0_3[5], eci0_3[6]]*1e-3 
    x0_4 = SA[eci0_4[1], eci0_4[2], eci0_4[3],eci0_4[4], eci0_4[5], eci0_4[6]]*1e-3 
    
    #Find state of orbit 1,2,3,4
    eci_1, t_hist1 = twobody_rk4(sat_dynamics, x0_1, T, h)
    eci_2, t_hist2 = twobody_rk4(sat_dynamics, x0_2, T, h)
    eci_3, t_hist3 = twobody_rk4(sat_dynamics, x0_3, T, h)
    eci_4, t_hist4 = twobody_rk4(sat_dynamics, x0_4, T, h)
    
    #6557 is the size of the array when it is at 1200. Need to make it a fixed size to be able to make it 
    #interactive
    
    size_eci1 = size(eci_1)
    size_eci2 = size(eci_2)
    size_eci3 = size(eci_3)
    size_eci4 = size(eci_4)
    
    eci_1_sized = zeros(6,6557)
    eci_2_sized = zeros(6,6557)
    eci_3_sized = zeros(6,6557)
    eci_4_sized = zeros(6,6557)
    
    eci_1_sized[1:size_eci1[1], 1:size_eci1[2]] = eci_1
    eci_2_sized[1:size_eci2[1], 1:size_eci2[2]] = eci_2
    eci_3_sized[1:size_eci3[1], 1:size_eci3[2]] = eci_3
    eci_4_sized[1:size_eci4[1], 1:size_eci4[2]] = eci_4
    
    centroid_guess = [(eci_1[1,1]+eci_2[1,1]+eci_3[1,1]+eci_4[1,1])/4, (eci_1[2,1]+eci_2[2,1]+eci_3[2,1]+eci_4[2,1])/4, (eci_1[3,1]+eci_2[3,1]+eci_3[3,1]+eci_4[3,1])/4] 
    onearth = sECEFtoGEOC(centroid_guess, use_degrees = true)
    geodetic = [onearth[1], onearth[2], 0]

    #Guess
    xyz = sGEOCtoECEF(geodetic, use_degrees = true)*1e-3
    
    return eci_1_sized, eci_2_sized, eci_3_sized, eci_4_sized, xyz
        
end

orbit_update (generic function with 1 method)

In [10]:
#Baseline
trueanom = 10
RAAN_sep = 2
delta_sep = 3
altitude = 1200

#initial
eci_1_s, eci_2_s, eci_3_s, eci_4_s, guess_s = orbit_update(trueanom, RAAN_sep, delta_sep, altitude);

# println("this is the size of eci_1_s: ", size(eci_1_s))
# println("this is the size of eci_2_s: ", size(eci_2_s))
# println("this is the size of eci_3_s: ", size(eci_3_s))
# println("this is the size of eci_4_s: ", size(eci_4_s))

In [11]:
Pin_read = readdlm("sat_cov_TOA_2freq.txt")

32×32 Matrix{Float64}:
  2.48895e-16  -9.03807e-18  -7.8636e-19   …   0.0           0.0
 -9.03807e-18   2.57148e-16  -4.57434e-18      0.0           0.0
 -7.8636e-19   -4.57434e-18   2.35897e-16      0.0           0.0
 -4.92783e-17  -4.34225e-16  -5.73703e-16      0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0          …   0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0          …   0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  ⋮                                        ⋱   ⋮            
  0.0           0.0           0.0          …   0.0           0.0
  0.0 

In [12]:
function measurment_residual(r0, satposes,zenith, t)
    
    omega = OMEGA_EARTH*time_scale
    
    scale_f1 = (40.3/(f1)^2)*1e22
    scale_f2 = (40.3/(f2)^2)*1e22
    
    angles = zeros(4)
    
    for i=1:4
        angles[i] = asind((R_EARTH*sind(zenith[i]))/(R_EARTH + hi))
    end
    
    A1 = [cos(omega*t[1]) sin(omega*t[1]) 0;
          -sin(omega*t[1]) cos(omega*t[1]) 0;
          0 0 1];
    A2 = [cos(omega*t[2]) sin(omega*t[2]) 0;
          -sin(omega*t[2]) cos(omega*t[2]) 0;
          0 0 1];
    A3 = [cos(omega*t[3]) sin(omega*t[3]) 0;
          -sin(omega*t[3]) cos(omega*t[3]) 0;
          0 0 1];
    A4 = [cos(omega*t[4]) sin(omega*t[4]) 0;
          -sin(omega*t[4]) cos(omega*t[4]) 0;
          0 0 1];
    A5 = [cos(omega*t[5]) sin(omega*t[5]) 0;
          -sin(omega*t[5]) cos(omega*t[5]) 0;
          0 0 1];
    A6 = [cos(omega*t[6]) sin(omega*t[6]) 0;
          -sin(omega*t[6]) cos(omega*t[6]) 0;
          0 0 1];
    A7 = [cos(omega*t[7]) sin(omega*t[7]) 0;
          -sin(omega*t[7]) cos(omega*t[7]) 0;
          0 0 1];
    A8 = [cos(omega*t[8]) sin(omega*t[8]) 0;
          -sin(omega*t[8]) cos(omega*t[8]) 0;
          0 0 1];
    
    res1 = norm(r0[1:3] - A1*satposes[1,1:3]) - c*(t[1] - r0[4]) + (scale_f1*(r0[5]/cosd(angles[1])))*(1e-3)/distance_scale;
    res2 = norm(r0[1:3] - A2*satposes[2,1:3]) - c*(t[2] - r0[4]) + (scale_f1*(r0[6]/cosd(angles[2])))*(1e-3)/distance_scale;
    res3 = norm(r0[1:3] - A3*satposes[3,1:3]) - c*(t[3] - r0[4]) + (scale_f1*(r0[7]/cosd(angles[3])))*(1e-3)/distance_scale;
    res4 = norm(r0[1:3] - A4*satposes[4,1:3]) - c*(t[4] - r0[4]) + (scale_f1*(r0[8]/cosd(angles[4])))*(1e-3)/distance_scale;
    res5 = norm(r0[1:3] - A5*satposes[1,1:3]) - c*(t[5] - r0[4]) + (scale_f2*(r0[5]/cosd(angles[1])))*(1e-3)/distance_scale;
    res6 = norm(r0[1:3] - A6*satposes[2,1:3]) - c*(t[6] - r0[4]) + (scale_f2*(r0[6]/cosd(angles[2])))*(1e-3)/distance_scale;
    res7 = norm(r0[1:3] - A7*satposes[3,1:3]) - c*(t[7] - r0[4]) + (scale_f2*(r0[7]/cosd(angles[3])))*(1e-3)/distance_scale;
    res8 = norm(r0[1:3] - A8*satposes[4,1:3]) - c*(t[8] - r0[4]) + (scale_f2*(r0[8]/cosd(angles[4])))*(1e-3)/distance_scale;
    
    return [res1, res2, res3, res4, res5, res6, res7, res8]
    
end

measurment_residual (generic function with 1 method)

In [13]:
function residual(x, sat_pose_t, zenith)
    
    omega = OMEGA_EARTH*time_scale
    
    angles = zeros(4)
    
    for i=1:4
        angles[i] = asind((R_EARTH*sind(zenith[i]))/(R_EARTH + hi))
    end

    scale_f1 = (40.3/(f1)^2)*1e22
    scale_f2 = (40.3/(f2)^2)*1e22
    
    A1 = [cos(omega*sat_pose_t[1,4]) sin(omega*sat_pose_t[1,4]) 0;
          -sin(omega*sat_pose_t[1,4]) cos(omega*sat_pose_t[1,4]) 0;
          0 0 1];
    A2 = [cos(omega*sat_pose_t[2,4]) sin(omega*sat_pose_t[2,4]) 0;
          -sin(omega*sat_pose_t[2,4]) cos(omega*sat_pose_t[2,4]) 0;
          0 0 1];
    A3 = [cos(omega*sat_pose_t[3,4]) sin(omega*sat_pose_t[3,4]) 0;
          -sin(omega*sat_pose_t[3,4]) cos(omega*sat_pose_t[3,4]) 0;
          0 0 1];
    A4 = [cos(omega*sat_pose_t[4,4]) sin(omega*sat_pose_t[4,4]) 0;
          -sin(omega*sat_pose_t[4,4]) cos(omega*sat_pose_t[4,4]) 0;
          0 0 1];
    A5 = [cos(omega*sat_pose_t[5,4]) sin(omega*sat_pose_t[5,4]) 0;
          -sin(omega*sat_pose_t[5,4]) cos(omega*sat_pose_t[5,4]) 0;
          0 0 1];
    A6 = [cos(omega*sat_pose_t[6,4]) sin(omega*sat_pose_t[6,4]) 0;
          -sin(omega*sat_pose_t[6,4]) cos(omega*sat_pose_t[6,4]) 0;
          0 0 1];
    A7 = [cos(omega*sat_pose_t[7,4]) sin(omega*sat_pose_t[7,4]) 0;
          -sin(omega*sat_pose_t[7,4]) cos(omega*sat_pose_t[7,4]) 0;
          0 0 1];
    A8 = [cos(omega*sat_pose_t[8,4]) sin(omega*sat_pose_t[8,4]) 0;
          -sin(omega*sat_pose_t[8,4]) cos(omega*sat_pose_t[8,4]) 0;
          0 0 1];
    
    res1 = norm(x[1:3] - A1*sat_pose_t[1,1:3]) - c*(sat_pose_t[1,4] - x[4]) + (scale_f1*(x[5]/cosd(angles[1])))*(1e-3)/distance_scale;
    res2 = norm(x[1:3] - A2*sat_pose_t[2,1:3]) - c*(sat_pose_t[2,4] - x[4]) + (scale_f1*(x[6]/cosd(angles[2])))*(1e-3)/distance_scale;
    res3 = norm(x[1:3] - A3*sat_pose_t[3,1:3]) - c*(sat_pose_t[3,4] - x[4]) + (scale_f1*(x[7]/cosd(angles[3])))*(1e-3)/distance_scale;
    res4 = norm(x[1:3] - A4*sat_pose_t[4,1:3]) - c*(sat_pose_t[4,4] - x[4]) + (scale_f1*(x[8]/cosd(angles[4])))*(1e-3)/distance_scale;
    res5 = norm(x[1:3] - A5*sat_pose_t[1,1:3]) - c*(sat_pose_t[5,4] - x[4]) + (scale_f2*(x[5]/cosd(angles[1])))*(1e-3)/distance_scale;
    res6 = norm(x[1:3] - A6*sat_pose_t[2,1:3]) - c*(sat_pose_t[6,4] - x[4]) + (scale_f2*(x[6]/cosd(angles[2])))*(1e-3)/distance_scale;
    res7 = norm(x[1:3] - A7*sat_pose_t[3,1:3]) - c*(sat_pose_t[7,4] - x[4]) + (scale_f2*(x[7]/cosd(angles[3])))*(1e-3)/distance_scale;
    res8 = norm(x[1:3] - A8*sat_pose_t[4,1:3]) - c*(sat_pose_t[8,4] - x[4]) + (scale_f2*(x[8]/cosd(angles[4])))*(1e-3)/distance_scale;
    
    return [res1, res2, res3, res4, res5, res6, res7, res8]
    
end

residual (generic function with 1 method)

In [14]:
function get_time(r0_TEC, satposes, zenith)
    
    n = 1000

    time = [zeros(8) for i = 1:1000]
    Rt = [zeros(8) for i = 1:1000]

    time[1] = [0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale] #initial guess

    Rt[1] = measurment_residual(r0_TEC, satposes, zenith, time[1])

    iters = 0

    for i=1:n

        Rt[i] = measurment_residual(r0_TEC, satposes, zenith, time[i])

        #println("this is norm of residual: ", norm(Rt[i]))
        
        iters += 1

        if(norm(Rt[i]) < 1e-6)
            break
        end

        jacobian = ForwardDiff.jacobian(dt -> measurment_residual(r0_TEC, satposes, zenith, dt), time[i])

        deltat = (jacobian)\-Rt[i]

        time[i+1]  = time[i] + deltat
    end
    
    Rt = Rt[1:iters]
    time = time[1:iters]
    time_measurment = time[end]
    
    return time_measurment
    
end

get_time (generic function with 1 method)

In [15]:
#frequency variables
f1 = 400e6 #in Hz
f2 = 600e6  #in Hz

6.0e8

In [16]:
function LS_Amatrix(x, y, scale, A, angle)
    #works
    #jacobian_pose = (x[1:3] - A'*y[1:3])/norm(y[1:3]-A*x[1:3])
    
    jacobian_pose = (x[1:3] - A*y[1:3])/norm(x[1:3]-A*y[1:3])
    
    #jacobian_pose = (x0[1:3] - r_sat[1:3])/norm(r_sat[1:3]-x0[1:3])
    
    dpdtau = c
    dpdTEC = (scale/cosd(angle))*(1e-3)/distance_scale
    
    return [jacobian_pose; dpdtau; dpdTEC]
end

LS_Amatrix (generic function with 1 method)

In [17]:
function LS_Amatrix_dpdy(x, y, scale, A, angle)
    
    #c = C_LIGHT
    
    #works
    jacobian_pose = (y[1:3] - A'*x[1:3])/norm(x[1:3]-A*y[1:3])
    
    dpdtau = -c
    
    return [jacobian_pose; dpdtau]
end

LS_Amatrix_dpdy (generic function with 1 method)

In [18]:
function customjacobian(initial_x0,sat_pose_t, elev_angles)
    c=1
    #println("this is initial x0: ", initial_x0)
    #println("this is sat pose t: ", sat_pose_t)
    #println("this is elev angles: ", elev_angles)
    
    while norm(residual(initial_x0, sat_pose_t, elev_angles))>1e-10
        
        #println("this is residual: ", residual(initial_x0, sat_pose_t, elev_angles))
        #println("this is residual: ", norm(residual(initial_x0, sat_pose_t, elev_angles)))
        omega = OMEGA_EARTH*time_scale


        angles = zeros(4)

        for i=1:4
            angles[i] = asind((R_EARTH*sind(elev_angles[i]))/(R_EARTH + hi))
        end

        scale_f1 = (40.3/(f1)^2)*1e22
        scale_f2 = (40.3/(f2)^2)*1e22

        A1 = [cos(omega*sat_pose_t[1,4]) sin(omega*sat_pose_t[1,4]) 0;
              -sin(omega*sat_pose_t[1,4]) cos(omega*sat_pose_t[1,4]) 0;
              0 0 1];
        A2 = [cos(omega*sat_pose_t[2,4]) sin(omega*sat_pose_t[2,4]) 0;
              -sin(omega*sat_pose_t[2,4]) cos(omega*sat_pose_t[2,4]) 0;
              0 0 1];
        A3 = [cos(omega*sat_pose_t[3,4]) sin(omega*sat_pose_t[3,4]) 0;
              -sin(omega*sat_pose_t[3,4]) cos(omega*sat_pose_t[3,4]) 0;
              0 0 1];
        A4 = [cos(omega*sat_pose_t[4,4]) sin(omega*sat_pose_t[4,4]) 0;
              -sin(omega*sat_pose_t[4,4]) cos(omega*sat_pose_t[4,4]) 0;
              0 0 1];
        A5 = [cos(omega*sat_pose_t[5,4]) sin(omega*sat_pose_t[5,4]) 0;
              -sin(omega*sat_pose_t[5,4]) cos(omega*sat_pose_t[5,4]) 0;
              0 0 1];
        A6 = [cos(omega*sat_pose_t[6,4]) sin(omega*sat_pose_t[6,4]) 0;
              -sin(omega*sat_pose_t[6,4]) cos(omega*sat_pose_t[6,4]) 0;
              0 0 1];
        A7 = [cos(omega*sat_pose_t[7,4]) sin(omega*sat_pose_t[7,4]) 0;
              -sin(omega*sat_pose_t[7,4]) cos(omega*sat_pose_t[7,4]) 0;
              0 0 1];
        A8 = [cos(omega*sat_pose_t[8,4]) sin(omega*sat_pose_t[8,4]) 0;
              -sin(omega*sat_pose_t[8,4]) cos(omega*sat_pose_t[8,4]) 0;
              0 0 1];

        P_computed1  = norm(initial_x0[1:3] - A1*sat_pose_t[1,1:3]) - c*(sat_pose_t[1,4] - initial_x0[4]) + (scale_f1*(initial_x0[5]/cosd(angles[1])))*(1e-3)/distance_scale;
        P_computed2  = norm(initial_x0[1:3] - A2*sat_pose_t[2,1:3]) - c*(sat_pose_t[2,4] - initial_x0[4]) + (scale_f1*(initial_x0[6]/cosd(angles[2])))*(1e-3)/distance_scale;
        P_computed3  = norm(initial_x0[1:3] - A3*sat_pose_t[3,1:3]) - c*(sat_pose_t[3,4] - initial_x0[4]) + (scale_f1*(initial_x0[7]/cosd(angles[3])))*(1e-3)/distance_scale;
        P_computed4  = norm(initial_x0[1:3] - A4*sat_pose_t[4,1:3]) - c*(sat_pose_t[4,4] - initial_x0[4]) + (scale_f1*(initial_x0[8]/cosd(angles[4])))*(1e-3)/distance_scale;
        P_computed5  = norm(initial_x0[1:3] - A5*sat_pose_t[5,1:3]) - c*(sat_pose_t[5,4] - initial_x0[4]) + (scale_f2*(initial_x0[5]/cosd(angles[1])))*(1e-3)/distance_scale;
        P_computed6  = norm(initial_x0[1:3] - A6*sat_pose_t[6,1:3]) - c*(sat_pose_t[6,4] - initial_x0[4]) + (scale_f2*(initial_x0[6]/cosd(angles[2])))*(1e-3)/distance_scale;
        P_computed7  = norm(initial_x0[1:3] - A7*sat_pose_t[7,1:3]) - c*(sat_pose_t[7,4] - initial_x0[4]) + (scale_f2*(initial_x0[7]/cosd(angles[3])))*(1e-3)/distance_scale;
        P_computed8  = norm(initial_x0[1:3] - A8*sat_pose_t[8,1:3]) - c*(sat_pose_t[8,4] - initial_x0[4]) + (scale_f2*(initial_x0[8]/cosd(angles[4])))*(1e-3)/distance_scale;

        #P_computed1 = norm(all_sats_scaled[1,1:3]-x0[1:3]) + c*x0[4] - c*all_sats_scaled[1,4]
        #P_computed2 = norm(all_sats_scaled[2,1:3]-x0[1:3]) + c*x0[4] - c*all_sats_scaled[2,4]
        #P_computed3 = norm(all_sats_scaled[3,1:3]-x0[1:3]) + c*x0[4] - c*all_sats_scaled[3,4]
        #P_computed4 = norm(all_sats_scaled[4,1:3]-x0[1:3]) + c*x0[4] - c*all_sats_scaled[4,4]

        P_computed = [P_computed1, P_computed2, P_computed3, P_computed4, P_computed5, P_computed6, P_computed7, P_computed8]

        P_observed = zeros(8)

        deltaP = P_observed - P_computed

        #println("this is delta P: ", norm(deltaP))
        A = zeros(8,8)

        A_row1 = LS_Amatrix(initial_x0, sat_pose_t[1,:], scale_f1, A1, angles[1])'
        A_row2 = LS_Amatrix(initial_x0, sat_pose_t[2,:], scale_f1, A2, angles[2])'
        A_row3 = LS_Amatrix(initial_x0, sat_pose_t[3,:], scale_f1, A3, angles[3])'
        A_row4 = LS_Amatrix(initial_x0, sat_pose_t[4,:], scale_f1, A4, angles[4])'
        A_row5 = LS_Amatrix(initial_x0, sat_pose_t[5,:], scale_f2, A5, angles[1])'
        A_row6 = LS_Amatrix(initial_x0, sat_pose_t[6,:], scale_f2, A6, angles[2])'
        A_row7 = LS_Amatrix(initial_x0, sat_pose_t[7,:], scale_f2, A7, angles[3])'
        A_row8 = LS_Amatrix(initial_x0, sat_pose_t[8,:], scale_f2, A8, angles[4])'

        #println("this is A_row1: ", A_row1)

        A[1,:] = [A_row1[1:5]; zeros(3)]
        A[2,:] = [A_row2[1:4];0.0; A_row2[5]; zeros(2)]
        A[3,:] = [A_row3[1:4];zeros(2); A_row3[5]; 0.0]
        A[4,:] = [A_row4[1:4];zeros(3); A_row4[5]]
        A[5,:] = [A_row5[1:5]; zeros(3)]
        A[6,:] = [A_row6[1:4];0.0; A_row6[5]; zeros(2)]
        A[7,:] = [A_row7[1:4];zeros(2); A_row7[5]; 0.0]
        A[8,:] = [A_row8[1:4];zeros(3); A_row8[5]]

        #println("this is the jacobian: ", A)

        #println(size(A))
    #     A = [A_row1 zeros(3)';
    #          A_row2[1:4] A_row2[5] zeros(3)']

        #; A_row2[1:4] 0 A_row2[5] zeros(3)'

    #     println(A_row1)
    #     println(A_row2)
    #     println(A)
    #     println(size(A))
    #     println(A_row2)
    #     println(A_row3)
    #     println(A_row4)

        #Need to reformat this A matrix
        #A = vcat(LS_Amatrix(x0, sat_pose_t[1,:], scale_f1, A1, angles[1])', LS_Amatrix(x0, sat_pose_t[2,:], scale_f1, angles[2])', LS_Amatrix(x0, sat_pose_t[3,:], scale_f1, angles[3])', LS_Amatrix(x0, sat_pose_t[4,:], scale_f1, angles[4])', 
        #         LS_Amatrix(x0, sat_pose_t[1,:], scale_f2, angles[1])', LS_Amatrix(x0, sat_pose_t[2,:], scale_f2, angles[2])', LS_Amatrix(x0, sat_pose_t[3,:], scale_f2, angles[3])', LS_Amatrix(x0, sat_pose_t[4,:], scale_f2, angles[4])')

        #println("solving QR decomp")
        F = qr(A)

        b = transpose(F.Q)*deltaP
        #println("solving system")
        deltax = F.R\b
        #println("deltax solved")
        #theresidual = residual(initial_x0, sat_pose_t, elev_angles)

        #println(theresidual)

        #println("this is the residual: ", norm(theresidual))


        #println("this is delta x: ", deltax)
        initial_x0 = initial_x0+deltax


    end

    x_sol = initial_x0
    #x_sol_rescaled = [x_sol[1]*distance_scale*1e3,x_sol[2]*distance_scale*1e3,x_sol[3]*distance_scale*1e3, x_sol[4]*time_scale, x_sol[5], x_sol[6], x_sol[7], x_sol[8]]
    
    return x_sol
    
end
#println("this is x_solution rescaled: ", x_sol_rescaled)

customjacobian (generic function with 1 method)

In [19]:
function createA(sat_pose_t, elev_angles, x_sol)
    
    omega = OMEGA_EARTH*time_scale #scaled

    angles = zeros(4)

    for i=1:4
        angles[i] = asind((R_EARTH*sind(elev_angles[i]))/(R_EARTH + hi))
    end

    scale_f1 = (40.3/(f1)^2)*1e22
    scale_f2 = (40.3/(f2)^2)*1e22

    A1 = [cos(omega*sat_pose_t[1,4]) sin(omega*sat_pose_t[1,4]) 0;
          -sin(omega*sat_pose_t[1,4]) cos(omega*sat_pose_t[1,4]) 0;
          0 0 1];
    A2 = [cos(omega*sat_pose_t[2,4]) sin(omega*sat_pose_t[2,4]) 0;
          -sin(omega*sat_pose_t[2,4]) cos(omega*sat_pose_t[2,4]) 0;
          0 0 1];
    A3 = [cos(omega*sat_pose_t[3,4]) sin(omega*sat_pose_t[3,4]) 0;
          -sin(omega*sat_pose_t[3,4]) cos(omega*sat_pose_t[3,4]) 0;
          0 0 1];
    A4 = [cos(omega*sat_pose_t[4,4]) sin(omega*sat_pose_t[4,4]) 0;
          -sin(omega*sat_pose_t[4,4]) cos(omega*sat_pose_t[4,4]) 0;
          0 0 1];
    A5 = [cos(omega*sat_pose_t[5,4]) sin(omega*sat_pose_t[5,4]) 0;
          -sin(omega*sat_pose_t[5,4]) cos(omega*sat_pose_t[5,4]) 0;
          0 0 1];
    A6 = [cos(omega*sat_pose_t[6,4]) sin(omega*sat_pose_t[6,4]) 0;
          -sin(omega*sat_pose_t[6,4]) cos(omega*sat_pose_t[6,4]) 0;
          0 0 1];
    A7 = [cos(omega*sat_pose_t[7,4]) sin(omega*sat_pose_t[7,4]) 0;
          -sin(omega*sat_pose_t[7,4]) cos(omega*sat_pose_t[7,4]) 0;
          0 0 1];
    A8 = [cos(omega*sat_pose_t[8,4]) sin(omega*sat_pose_t[8,4]) 0;
          -sin(omega*sat_pose_t[8,4]) cos(omega*sat_pose_t[8,4]) 0;
          0 0 1];

    #create dfdy
    dfdx = zeros(8,8)

    dfdx_row1 = LS_Amatrix(x_sol, sat_pose_t[1,:], scale_f1, A1, angles[1])'
    dfdx_row2 = LS_Amatrix(x_sol, sat_pose_t[2,:], scale_f1, A2, angles[2])'
    dfdx_row3 = LS_Amatrix(x_sol, sat_pose_t[3,:], scale_f1, A3, angles[3])'
    dfdx_row4 = LS_Amatrix(x_sol, sat_pose_t[4,:], scale_f1, A4, angles[4])'
    dfdx_row5 = LS_Amatrix(x_sol, sat_pose_t[5,:], scale_f2, A5, angles[1])'
    dfdx_row6 = LS_Amatrix(x_sol, sat_pose_t[6,:], scale_f2, A6, angles[2])'
    dfdx_row7 = LS_Amatrix(x_sol, sat_pose_t[7,:], scale_f2, A7, angles[3])'
    dfdx_row8 = LS_Amatrix(x_sol, sat_pose_t[8,:], scale_f2, A8, angles[4])'

    dfdx[1,:] = [dfdx_row1[1:5]; zeros(3)]
    dfdx[2,:] = [dfdx_row2[1:4];0.0; dfdx_row2[5]; zeros(2)]
    dfdx[3,:] = [dfdx_row3[1:4];zeros(2); dfdx_row3[5]; 0.0]
    dfdx[4,:] = [dfdx_row4[1:4];zeros(3); dfdx_row4[5]]
    dfdx[5,:] = [dfdx_row5[1:5]; zeros(3)]
    dfdx[6,:] = [dfdx_row6[1:4];0.0; dfdx_row6[5]; zeros(2)]
    dfdx[7,:] = [dfdx_row7[1:4];zeros(2); dfdx_row7[5]; 0.0]
    dfdx[8,:] = [dfdx_row8[1:4];zeros(3); dfdx_row8[5]]

    #derivative wrt measurements (satellite position and times)
    dfdy = zeros(8,32)

    #create dfdx
    dfdy[1, 1:4] = LS_Amatrix_dpdy(x_sol, sat_pose_t[1,:], scale_f1, A1, angles[1])'[1:4]
    dfdy[2, 5:8] = LS_Amatrix_dpdy(x_sol, sat_pose_t[2,:], scale_f1, A2, angles[2])'[1:4]
    dfdy[3, 9:12] = LS_Amatrix_dpdy(x_sol, sat_pose_t[3,:], scale_f1, A3, angles[3])'[1:4]
    dfdy[4, 13:16] = LS_Amatrix_dpdy(x_sol, sat_pose_t[4,:], scale_f1, A4, angles[4])'[1:4]
    dfdy[5, 17:20] = LS_Amatrix_dpdy(x_sol, sat_pose_t[5,:], scale_f2, A5, angles[1])'[1:4]
    dfdy[6, 21:24] = LS_Amatrix_dpdy(x_sol, sat_pose_t[6,:], scale_f2, A6, angles[2])'[1:4]
    dfdy[7, 25:28] = LS_Amatrix_dpdy(x_sol, sat_pose_t[7,:], scale_f2, A7, angles[3])'[1:4]
    dfdy[8, 29:32] = LS_Amatrix_dpdy(x_sol, sat_pose_t[8,:], scale_f2, A8, angles[4])'[1:4]

    A = -inv(dfdx)*dfdy
    
    return A
end

createA (generic function with 1 method)

In [20]:
#scaled
using Distributions
mu = 8e-6
sigma = 3e-6
lb = 3e-6
ub = 13e-6
d = Truncated(Normal(mu,sigma), lb, ub)

hi = 350e3 #in meters

350000.0

In [21]:
function calculate_covariance(sat_poses, elev_angles, guess)
    TEC = rand(d,4) 
    r0_TEC= vec([r0_scaled TEC])
    sat_poses_scaled = sat_poses/distance_scale #scale the data
    t = get_time(r0_TEC, sat_poses_scaled, elev_angles)
    sat_poses_2 = vcat(sat_poses_scaled, sat_poses_scaled)
    sat_pose_t = [sat_poses_2 t]
    
    #println(size(sat_pose_t))
    
    #println("good so far")
    initial_x0 = [guess[1]/distance_scale, guess[2]/distance_scale, guess[3]/distance_scale, 0.0001/time_scale, 3e-6, 3.5e-6, 2e-6, 4e-6]
    
    #println("solving for initial x0")
    
    x_sol = customjacobian(initial_x0,sat_pose_t, elev_angles)
    
    #println("soved for x_sol")
    
    A = createA(sat_pose_t, elev_angles, x_sol)
    
    covariance_tag = A*Pin_read*transpose(A)
    
    pose_covariance = covariance_tag[1:3, 1:3]
    
    pose_covariance_scaled = covariance_tag[1:3, 1:3]*(1e3*distance_scale)^2
    
    PDOP_LS = sqrt(covariance_tag[1,1] + covariance_tag[2,2] + covariance_tag[3,3])

    PDOP_LS_scaled = PDOP_LS*distance_scale*1e3
    
    return PDOP_LS_scaled, pose_covariance_scaled
    
end

calculate_covariance (generic function with 1 method)

In [22]:
#DRAW THE ELLIPSE
function draw_ellipse(covariance_matrix)
    
    eig_value = eigvals(covariance_matrix)
    eig_vecs = eigvecs(covariance_matrix)
    
    max_eigval_index = argmax(eig_value)
    min_eigval_index = argmin(eig_value)

    max_eigval = eig_value[max_eigval_index]
    min_eigval = eig_value[min_eigval_index]
    
    max_eigvec = eig_vecs[:, max_eigval_index]
    min_eigvec = eig_vecs[:, min_eigval_index]
    
    angle = atan(max_eigvec[2], max_eigvec[1])
    
    if angle<0
        angle = angle + 2*pi;
    end
    
    chisquare_val = 2.4477
    theta_grid = 0:0.01:2*pi

    phi = angle;
    #X0= avg
    #Y0 = avg
    a = chisquare_val*sqrt(max_eigval)
    b = chisquare_val*sqrt(min_eigval)

    ellipse_x = zeros(size(theta_grid)[1], 1)
    ellipse_y = zeros(size(theta_grid)[1], 1)

    for i=1:size(theta_grid)[1]

        #Get ellipse coordinates 
        ellipse_x[i,1] = a*cos(theta_grid[i])
        ellipse_y[i,1] = b*sin(theta_grid[i])

    end
    
    R = [cos(phi) sin(phi); -sin(phi) cos(phi)];
    r_ellipse = [ellipse_x ellipse_y]*R
    
    return r_ellipse

        
end

draw_ellipse (generic function with 1 method)

In [51]:
#LOAD the Figure

#scene = Scene(backgroundcolor=:black, show_axis=true, resolution=(2048,1024))

#earth_mesh = mesh(scene, Sphere(Point3f0(0), 6371f0), color= earth)

#F = Figure(backgroundcolor = :black)
#ax = Axis(F[1,1], backgroundcolor = :transparent)


#mesh!(Sphere(Point3f0(0), 6371f0), color= earth)
    
    
    #println("this is svalues: ", svalues)

#f = Figure()
#f = Figure(resolution=(2048,1024))
#Axis(f[1, 1], title = "Interactive Orbit Plot")

#working before

earth = load(download("https://svs.gsfc.nasa.gov/vis/a000000/a002900/a002915/bluemarble-2048.png"));

#f = Figure()
#f = Figure(resolution = (2048,1024))


#allows user zoom and drag
#fig, ax, earth = mesh(Sphere(Point3f0(0), 6371f0), color= earth, figure = (resolution = (1000, 1000),))

#working
fig, ax, earth = mesh(Sphere(Point3f0(0), 6371f0), color= earth, figure = (resolution = (1200,1200),))

#pos = f[1, 1]
#mesh(pos, Sphere(Point3f0(0), 6371f0), color= earth, figure = (resolution = (2048,1024),))


#ellipse_fig = f[2,3]
ellipse_fig = fig[2,2]


#works before. Fixes 3D 
# fig, axis, earth = mesh(Sphere(Point3f0(0), 6371f0), color= earth, figure = (resolution = (1000, 1000),), 
# axis = (; type = Axis3, protrusions = (0, 0, 0, 0),
#         viewmode = :fit, limits = (-7000, 7000, -7000, 7000, -7000, 7000)))

# fig, axis, earth = mesh(Sphere(Point3f0(0), 6371f0), color= earth, figure = (resolution = (1000, 1000),) 
# )



#, limits = (-7000, 7000, -7000, 7000, -7000, 7000)



#wireframe!(axis, Sphere(Point3f0(0), 6371f0), color=(:black, 0.2), linewidth=2, transparency=true)

#fig = Figure();

#ax = Axis(fig[1,1])

#mesh!(Sphere(Point3f0(0), 6371f0), color= earth, figure = (resolution = (1000, 1000),))

along = Observable(Int(1))
ellipse_xy1 = Observable(zeros(629))
ellipse_xy2 = Observable(zeros(629))

ellipse_xz1 = Observable(zeros(629))
ellipse_xz2 = Observable(zeros(629))

ellipse_yz1 = Observable(zeros(629))
ellipse_yz2 = Observable(zeros(629))

eci_1_x = Observable(eci_1_s[1,:])
eci_2_x = Observable(eci_2_s[1,:])
eci_3_x = Observable(eci_3_s[1,:])
eci_4_x = Observable(eci_4_s[1,:])

eci_1_y = Observable(eci_1_s[2,:])
eci_2_y = Observable(eci_2_s[2,:])
eci_3_y = Observable(eci_3_s[2,:])
eci_4_y = Observable(eci_4_s[2,:])

eci_1_z = Observable(eci_1_s[3,:])
eci_2_z = Observable(eci_2_s[3,:])
eci_3_z = Observable(eci_3_s[3,:])
eci_4_z = Observable(eci_4_s[3,:])

sats = Observable([Point3f0(eci_1_s[1,1],eci_1_s[2,1],eci_1_s[3,1]), 
                   Point3f0(eci_2_s[1,1],eci_2_s[2,1],eci_2_s[3,1]),
                   Point3f0(eci_3_s[1,1],eci_3_s[2,1],eci_3_s[3,1]),
                   Point3f0(eci_4_s[1,1],eci_4_s[2,1],eci_4_s[3,1])])

# sats = Observable([Point3f0(eci_1_s[1,along],eci_1_s[2,along],eci_1_s[3,along]), 
#                    Point3f0(eci_2_s[1,along],eci_2_s[2,along],eci_2_s[3,along]),
#                    Point3f0(eci_3_s[1,along],eci_3_s[2,along],eci_3_s[3,along]),
#                    Point3f0(eci_4_s[1,along],eci_4_s[2,along],eci_4_s[3,along])])


guess = Observable(Point3f0(eci_1_s[1,1],eci_1_s[2,1],eci_1_s[3,1]))

lsgrid = labelslidergrid!(
    fig,
    ["Orbit Altitude", "RAAN Seperation", "True Anomaly Seperation", "Delta True Anomaly Seperation", "Time Step"],
    [400:100:1200, 1:1:5, 1:0.5:15, 1:1:5, 1:10:1000];
    formats = [x -> "$(round(x, digits = 1))$s" for s in ["km", "Degrees", "Degrees", "Degrees", " "]],
    width = 500,
    tellheight = false)

#this puts the result on the bottom 
#fig[2, 1] = lsgrid.layout

fig[1,2] = lsgrid.layout

#Label(fig[1, 2], "Parameters", padding = (0, 0, 1, 0))

#f[1,3] = lsgrid.layout

sliderobservables = [s.value for s in lsgrid.sliders]
bars = lift(sliderobservables...) do slvalues...
    values = [slvalues...]
    altitude = values[1]
    RAAN_sep = values[2]
    true_anom = values[3]
    delta_true_anom = values[4]
    alongorbit = values[5]
    
    #println("this is values; ", values)
    
    eci_1_update, eci_2_update, eci_3_update, eci_4_update, xyz_ = orbit_update(true_anom, RAAN_sep, delta_true_anom, altitude);
    
    
    #covariance = calculate_covariance(sat_poses)
    
    #println("this is type: ", typeof(eci_1))
    eci_1_x[] = eci_1_update[1,:]
    eci_2_x[] = eci_2_update[1,:]
    eci_3_x[] = eci_3_update[1,:]
    eci_4_x[] = eci_4_update[1,:]
    
    eci_1_y[] = eci_1_update[2,:]
    eci_2_y[] = eci_2_update[2,:]
    eci_3_y[] = eci_3_update[2,:]
    eci_4_y[] = eci_4_update[2,:]
    
    eci_1_z[] = eci_1_update[3,:]
    eci_2_z[] = eci_2_update[3,:]
    eci_3_z[] = eci_3_update[3,:]
    eci_4_z[] = eci_4_update[3,:]
    
    along[] = alongorbit 
    
    intorbit = Int(alongorbit)
    
    #sat_poses is in km
    sat_poses = [eci_1_update[1,intorbit] eci_1_update[2,intorbit] eci_1_update[3,intorbit]; 
                 eci_2_update[1,intorbit] eci_2_update[2,intorbit] eci_2_update[3,intorbit];
                 eci_3_update[1,intorbit] eci_3_update[2,intorbit] eci_3_update[3,intorbit];
                 eci_4_update[1,intorbit] eci_4_update[2,intorbit] eci_4_update[3,intorbit]]
    
    #println("this is sat poses: ", sat_poses)
    sats[] = [Point3f0(eci_1_update[1,intorbit],eci_1_update[2,intorbit],eci_1_update[3,intorbit]), 
                   Point3f0(eci_2_update[1,intorbit],eci_2_update[2,intorbit],eci_2_update[3,intorbit]),
                   Point3f0(eci_3_update[1,intorbit],eci_3_update[2,intorbit],eci_3_update[3,intorbit]),
                   Point3f0(eci_4_update[1,intorbit],eci_4_update[2,intorbit],eci_4_update[3,intorbit])]
    
    
    #println("this is sats 1: ", eci_1_update[1, intorbit])
    centroid_guess = [(eci_1_update[1,intorbit]+eci_2_update[1,intorbit]+eci_3_update[1,intorbit]+eci_4_update[1,intorbit])/4,
                      (eci_1_update[2,intorbit]+eci_2_update[2,intorbit]+eci_3_update[2,intorbit]+eci_4_update[2,intorbit])/4,
                      (eci_1_update[3,intorbit]+eci_2_update[3,intorbit]+eci_3_update[3,intorbit]+eci_4_update[3,intorbit])/4] 
    
    onearth = sECEFtoGEOC(centroid_guess, use_degrees = true)
    geodetic = [onearth[1], onearth[2], 0]

    #Guess
    xyz = sGEOCtoECEF(geodetic, use_degrees = true)*1e-3
    
    #sat poses is in km. need to switch to meters to find elev angle
    elev_angles = elevation_angle(sat_poses*1e3, r0[1:3]*1e3)
    
    #println("these are the elevation angles: ", elev_angles)
    inhorizon = all(x->x<70, elev_angles)
    if inhorizon==false
        println("Not in the horizon")
        #continue #skip the current iteration
    else 
        #sat poses is in km, xyz is in km
        PDOP, pose_covariance = calculate_covariance(sat_poses, elev_angles, xyz)
        println("this is PDOP: ", PDOP)
        
        pose_covariance_xz = [pose_covariance[1,1] pose_covariance[1,3];pose_covariance[3,1] pose_covariance[3,3]]
        
        r_ellipse_xy = draw_ellipse(pose_covariance[1:2, 1:2])
        r_ellipse_xz = draw_ellipse(pose_covariance_xz)
        r_ellipse_yz = draw_ellipse(pose_covariance[2:3, 2:3])
        
        #println("this is r_ellipse: ", r_ellipse)
        ellipse_xy1[] = r_ellipse_xy[:,1]
        ellipse_xy2[] = r_ellipse_xy[:,2]
        
        ellipse_xz1[] = r_ellipse_xz[:,1]
        ellipse_xz2[] = r_ellipse_xz[:,2]
        
        ellipse_yz1[] = r_ellipse_yz[:,1]
        ellipse_yz2[] = r_ellipse_yz[:,2]
        
    end
    
    
    #println("this is xyz: ", xyz)
    
    guess[] = xyz
    
    end
    
set_close_to!(lsgrid.sliders[1], 1200)
set_close_to!(lsgrid.sliders[2], 2)
set_close_to!(lsgrid.sliders[3], 10)
set_close_to!(lsgrid.sliders[4], 3)
set_close_to!(lsgrid.sliders[5], 1)


#Plot tag position
tagpose = meshscatter!([r0[1]],[r0[2]],[r0[3]], markersize = 150, color=:red, label = "Tag")

#Plot the Guess
guesspose = meshscatter!(guess, markersize = 150, color=:brown, label= "Guess")

#Plot satellites
satpose = meshscatter!(sats, markersize = 150, color=:yellow, label= "Satellites")

#Good plotting
#Plot orbit  of sat 1/2
orbit12 = lines!(eci_1_x, eci_1_y, eci_1_z, color = :blue, label= "Orbit 1/2")

#Plot orbit of sat 3/4
orbit34 = lines!(eci_3_x, eci_3_y, eci_3_z, color = :purple, label= "Orbit 3/4")

ellipse1= lines(ellipse_fig[1,1], ellipse_xy1, ellipse_xy2)
ellipse2 = lines(ellipse_fig[2,1], ellipse_xz1, ellipse_xz2)
ellipse3 = lines(ellipse_fig[1,2], ellipse_yz1, ellipse_yz2)

#axes1 = Axis(ellipse_fig[1, 1])

#xlims!(ax[2,3], [-200, 200]) # as vector

Label(ellipse_fig[1, 1, Top()], "Covariance XY", padding = (0, 0, 10, 0))
Label(ellipse_fig[2, 1, Top()], "Covariance XZ", padding = (0, 0, 10, 0))
Label(ellipse_fig[1, 2, Top()], "Covariance YZ", padding = (0, 0, 10, 0))

#result = text!([Point3f0(0, 0, 8000)]; text = "Result")


#f[1, 2] = Legend(fig, ax, "Trig Functions", framevisible = false)

# Legend(fig[1, 1],
#     [orbit12, orbit34, tagpose, guesspose, satpose],
#     ["Orbit 1/2", "Orbit 3/4", "Tag", "Initial Guess", "Satellites"])



display(fig)

Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon


GLMakie.Screen(...)

Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
this is PDOP: 62.50629311245286
this is PDOP: 59.87323711943094
this is PDOP: 58.18289815332669
this is PDOP: 57.49918365983398
this is PDOP: 58.492520953795555
this is PDOP: 63.248092316797134
Not in the horizon
this is PDOP: 63.248092316797276
this is PDOP: 57.49918365983423
this is PDOP: 58.18289815332669
this is PDOP: 57.39629164205686
this is PDOP: 57.49918365983461
this is PDOP: 58.49252095379526
this is PDOP: 60.393644668029026
Not in the horizon
this is PDOP: 62.50629311245238
this is PDOP: 59.873237119430826
this is PDOP: 57.49918365983448
this is PDOP: 63.24809231679723
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the hor

In [ ]:
A = [1 1 1;2 2 2]

In [ ]:
b = [5,6,7,8]

In [ ]:
J = vcat(A,A)

In [ ]:
[Jb]

In [1]:
theta_grid = 0:0.01:2*pi

0.0:0.01:6.28